In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import random
import pandas as pd
import torch
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]

    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, train_val_seed=1337):
    class_num = 41 # NOTE: pre-computed, should not need change
    mode = 'train' if (split == 'train' or split == 'val') else 'test'

    label_dict = {}
    if mode != 'test':
      phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

      for line in phone_file:
          line = line.strip('\n').split(' ')
          label_dict[line[0]] = [int(p) for p in line[1:]]

    if split == 'train' or split == 'val':
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(train_val_seed)
        random.shuffle(usage_list)
        percent = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:percent] if split == 'train' else usage_list[percent:]
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode != 'test':
      y = torch.empty(max_len, concat_nframes, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode != 'test':
          label = torch.LongTensor(label_dict[fname]).unsqueeze(1)
          label = concat_feat(label, concat_nframes)

        X[idx: idx + cur_len, :] = feat
        if mode != 'test':
          y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode != 'test':
      y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode != 'test':
      print(y.shape)
      return X, y
    else:
      return X

In [3]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx].view(-1, 39), self.label[idx]
        else:
            return self.data[idx].view(-1, 39)

    def __len__(self):
        return len(self.data)

In [4]:
! pip install pytorch-crf

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchcrf import CRF


#class BiLSTM_CRF(nn.Module):
#
#    def __init__(self, class_size=41, input_dim=39, hidden_dim=128,dropout=0.5):
#        super().__init__()
#        self.input_dim = input_dim
#        self.hidden_dim = hidden_dim
#        self.class_size = class_size
#
#        self.lstm = nn.LSTM(input_dim, hidden_dim // 2, dropout=dropout,
#                            num_layers=3, bidirectional=True, batch_first=True)
#        self.hidden2tag = nn.Sequential(
#            nn.Dropout(dropout),
#            nn.Linear(hidden_dim, class_size)
#        )
#
#        self.crf = CRF(self.class_size, batch_first=True)
#        
#    def get_emissions(self, x):
#        feats, _ = self.lstm(x)
#        return self.hidden2tag(feats)
#
#    def likelihood(self, x, y):
#        emissions = self.get_emissions(x)
#        loss = self.crf(emissions, y)
#        return loss
#
#    def forward(self, x):  # dont confuse this with _forward_alg above.
#        emissions = self.get_emissions(x)
#        seqs = self.crf.decode(emissions)
#        return torch.LongTensor(seqs)

class BiLSTM(nn.Module):
    def __init__(self, class_size=41, input_dim=39, hidden_dim=192, dropout=0.5):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.class_size = class_size
        self.lstm = nn.LSTM(input_dim, hidden_dim // 2, dropout=dropout,
                            num_layers=3, bidirectional=True, batch_first=True)
        self.hidden2tag = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, class_size)
        )
        
    def forward(self, x):
        feats, _ = self.lstm(x)
        return self.hidden2tag(feats)
    
class Crf(nn.Module):
    def __init__(self, class_size=41):
        super().__init__()
        self.class_size = class_size
        self.crf = CRF(self.class_size, batch_first=True)
        
    def likelihood(self, x, y):
        return self.crf(x, y)
    
    def forward(self, x):
        return torch.LongTensor(self.crf.decode(x))

In [6]:
# data prarameters
concat_nframes = 21          # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
mid = concat_nframes//2
train_ratio = 0.95               # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 0                        # random seed
batch_size = 2048                # batch size
num_epoch = 50                   # the number of training epoch
early_stopping = 8
learning_rate = 0.0001            #learning rate
model1_path = './model1.ckpt'     # the path where the checkpoint will be saved
model2_path = './model2.ckpt'
# model parameters
input_dim = 39 * concat_nframes # the input dim of the model, you should not change the value
hidden_layers = 3              # the number of hidden layers
hidden_dim = 1024              # the hidden dim

In [7]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

DEVICE: cuda:0


In [8]:
import numpy as np

#fix seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [9]:
import gc

# preprocess data
train_X, train_y = preprocess_data(split='train', feat_dir='../input/ml2022spring-hw2/libriphone/libriphone/feat', 
                                   phone_path='../input/ml2022spring-hw2/libriphone/libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
val_X, val_y = preprocess_data(split='val', feat_dir='../input/ml2022spring-hw2/libriphone/libriphone/feat', 
                               phone_path='../input/ml2022spring-hw2/libriphone/libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)

# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for train: 4071


4071it [00:42, 95.04it/s]


[INFO] train set
torch.Size([2513163, 819])
torch.Size([2513163, 21])
[Dataset] - # phone classes: 41, number of utterances for val: 215


215it [00:02, 95.30it/s]


[INFO] val set
torch.Size([130995, 819])
torch.Size([130995, 21])


In [10]:
# fix random seed
same_seeds(seed)

# create model, define a loss function, and optimizer
#model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
#model = BiLSTM_CRF().to(device)
bilstm = BiLSTM().to(device)
crf = Crf().to(device)
optimizer1 = torch.optim.AdamW(bilstm.parameters(), lr=learning_rate*20, weight_decay=0.015)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer1, 
                                        T_0=8, T_mult=2, eta_min=learning_rate/2)

optimizer2  = torch.optim.AdamW(crf.parameters(), lr=learning_rate*500, weight_decay=1e-8)

total_num = 0
for i, param in enumerate(bilstm.parameters()):
    print('Layer:', i, '    parameter num:',param.numel(), '    shape:', param.shape)
    total_num += param.numel()

print(f'Total parameters num: {total_num}')

total_num = 0
for i, param in enumerate(crf.parameters()):
    print('Layer:', i, '    parameter num:',param.numel(), '    shape:', param.shape)
    total_num += param.numel()

print(f'Total parameters num: {total_num}')

Layer: 0     parameter num: 14976     shape: torch.Size([384, 39])
Layer: 1     parameter num: 36864     shape: torch.Size([384, 96])
Layer: 2     parameter num: 384     shape: torch.Size([384])
Layer: 3     parameter num: 384     shape: torch.Size([384])
Layer: 4     parameter num: 14976     shape: torch.Size([384, 39])
Layer: 5     parameter num: 36864     shape: torch.Size([384, 96])
Layer: 6     parameter num: 384     shape: torch.Size([384])
Layer: 7     parameter num: 384     shape: torch.Size([384])
Layer: 8     parameter num: 73728     shape: torch.Size([384, 192])
Layer: 9     parameter num: 36864     shape: torch.Size([384, 96])
Layer: 10     parameter num: 384     shape: torch.Size([384])
Layer: 11     parameter num: 384     shape: torch.Size([384])
Layer: 12     parameter num: 73728     shape: torch.Size([384, 192])
Layer: 13     parameter num: 36864     shape: torch.Size([384, 96])
Layer: 14     parameter num: 384     shape: torch.Size([384])
Layer: 15     parameter num: 3

In [11]:
best_acc = 0.0
early_stop_count = 0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    train_item =0
    # training
    bilstm.train() # set the model to training mode
    crf.train()
    pbar = tqdm(train_loader, ncols=110)
    pbar.set_description(f'T: {epoch+1}/{num_epoch}')
    samples = 0
    for i, batch in enumerate(pbar):
        features, labels = batch
        features, labels = features.to(device), labels.to(device)
        
        optimizer1.zero_grad() 
        optimizer2.zero_grad()
        loss = -crf.likelihood(bilstm(features), labels)
        loss.backward()
        grad_norm = nn.utils.clip_grad_norm_(bilstm.parameters(), max_norm=50)
        optimizer1.step()
        optimizer2.step()
        
        train_loss += loss.item()
        train_item += labels.size(0)
        
        lr1 = optimizer1.param_groups[0]["lr"]
        lr2 = optimizer2.param_groups[0]["lr"]
        pbar.set_postfix({'lr1':lr1, 'lr2':lr2, 'loss':train_loss/train_item})
    scheduler.step()
    pbar.close()
    # validation
    if len(val_set) > 0:
        bilstm.eval() # set the model to evaluation mode
        crf.eval()
        with torch.no_grad():
            pbar = tqdm(val_loader, ncols=110)
            pbar.set_description(f'V: {epoch+1}/{num_epoch}')
            samples = 0
            for i, batch in enumerate(pbar):
                features, labels = batch
                features, labels = features.to(device), labels.to(device)
                outputs = crf(bilstm(features))                
                val_acc += (outputs[:, mid] == labels[:, mid].cpu()).sum().item()
                samples += labels.size(0)
                pbar.set_postfix({'val acc':val_acc/samples})
            pbar.close()
            # if the model improves, save a checkpoint at this epoch
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(bilstm.state_dict(), model1_path)
            torch.save(crf.state_dict(), model2_path)
            print('saving model with acc {:.3f}'.format(best_acc/(len(val_set))))
            early_stop_count = 0
        else:
            early_stop_count += 1
            if early_stop_count >= early_stopping:
                print(f"Epoch: {epoch + 1}, model not improving, early stopping.")
                break

T: 1/50:   0%|                                                                       | 0/1228 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/torchcrf/__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  /usr/local/src/pytorch/aten/src/ATen/native/TensorCompare.cpp:255.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)
V: 1/50: 100%|█████████████████████████████████████████████████| 64/64 [00:53<00:00,  1.20it/s, val acc=0.751]


saving model with acc 0.751


V: 2/50: 100%|█████████████████████████████████████████████████| 64/64 [00:52<00:00,  1.21it/s, val acc=0.765]


saving model with acc 0.765


V: 3/50: 100%|█████████████████████████████████████████████████| 64/64 [00:54<00:00,  1.18it/s, val acc=0.778]


...

saving model with acc 0.793


V: 22/50: 100%|████████████████████████████████████████████████| 64/64 [00:51<00:00,  1.23it/s, val acc=0.794]


saving model with acc 0.794


V: 30/50: 100%|████████████████████████████████████████████████| 64/64 [00:51<00:00,  1.24it/s, val acc=0.783]

Epoch: 30, model not improving, early stopping.


In [12]:
del train_loader, val_loader
gc.collect()

21

In [13]:
# load data
test_X = preprocess_data(split='test', feat_dir='../input/ml2022spring-hw2/libriphone/libriphone/feat',
                         phone_path='../input/ml2022spring-hw2/libriphone/libriphone', concat_nframes=concat_nframes)

test_set = LibriDataset(test_X)

import gc
del test_X
gc.collect()

# get dataloader
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 1078


1078it [00:12, 86.48it/s]


[INFO] test set
torch.Size([646268, 819])


In [14]:
# load model
#model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
bilstm = BiLSTM().to(device)
bilstm.load_state_dict(torch.load(model1_path))

crf = Crf().to(device)
crf.load_state_dict(torch.load(model2_path))

<All keys matched successfully>

In [15]:
pred = np.array([], dtype=np.int32)

bilstm.eval()
crf.eval()
with torch.no_grad():
    for features in tqdm(test_loader):
        features = features.to(device)
        outputs = crf(bilstm(features))
        pred = np.concatenate((pred, outputs.detach().cpu()[:, mid]), axis=0)

100%|██████████| 316/316 [04:20<00:00,  1.21it/s]


In [16]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))